# Step 5: Multi-Objective Ranking and Network Selection

**Rank and select the best network from 20 candidates**

This notebook:
1. Loads 20 networks with space syntax metrics from Step 4
2. Defines ranking criteria based on reference cities
3. Computes multi-objective scores for each network
4. Ranks networks and selects the top candidate
5. Visualizes ranking results and selected network
6. Saves selected network for final building generation

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from pathlib import Path
import pickle
from scipy import stats

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

print("✓ Libraries loaded")

## Configuration

In [ ]:
WINDOW_SIZE_M = 500  # 500×500m window

# Create output directories
Path("outputs/generated/visualizations").mkdir(parents=True, exist_ok=True)
Path("outputs/generated/selected").mkdir(parents=True, exist_ok=True)

print(f"Window size: {WINDOW_SIZE_M}m × {WINDOW_SIZE_M}m")
print("✓ Output directories created")

## Load Data

In [ ]:
# Load reference city data
with open('outputs/data/reference_cities_data.pkl', 'rb') as f:
    reference_data = pickle.load(f)

# Load generated networks with space syntax from Step 4
with open('outputs/generated/syntax/networks_with_syntax_20.pkl', 'rb') as f:
    generated_networks = pickle.load(f)

print("✓ Loaded reference data from Step 1")
print(f"✓ Loaded {len(generated_networks)} networks from Step 4")

# Reference city for comparison (London)
reference_city = 'london'
print(f"\nUsing {reference_city.upper()} as primary reference for ranking")

## Define Ranking Criteria

In [ ]:
# Extract reference metrics
ref_data = reference_data[reference_city]

ref_metrics = {
    # Network morphology (divided by 2 for doubled roads)
    'node_density': ref_data['morphology']['node_density'] / 2,
    'avg_degree': ref_data['morphology']['avg_degree'] / 2,
    'avg_segment_length': ref_data['morphology']['avg_segment_length'],  # NOT doubled
    
    # Space syntax
    'intelligibility': ref_data['syntax']['intelligibility'],
    'mean_depth': ref_data['syntax']['mean_depth']
}

print("Reference Metrics (Target Values):")
print("="*50)
for metric, value in ref_metrics.items():
    print(f"  {metric:25s}: {value:.3f}")
print("="*50)

# Define weights for each criterion (sum to 1.0)
weights = {
    'node_density': 0.15,
    'avg_degree': 0.15,
    'avg_segment_length': 0.20,
    'intelligibility': 0.30,
    'mean_depth': 0.20
}

print("\nRanking Weights:")
print("="*50)
for metric, weight in weights.items():
    print(f"  {metric:25s}: {weight:.2f}")
print(f"  {'TOTAL':25s}: {sum(weights.values()):.2f}")
print("="*50)

## Compute Scores for All Networks

In [ ]:
def compute_similarity_score(value, target, metric_name=''):
    """
    Compute similarity score between value and target.
    Returns score in [0, 1] where 1 = perfect match.
    
    Uses inverse of normalized absolute difference.
    """
    if target == 0:
        return 1.0 if value == 0 else 0.0
    
    # Normalized absolute difference
    diff = abs(value - target) / target
    
    # Convert to similarity score (exponential decay)
    # score = exp(-k * diff) where k controls sensitivity
    k = 2.0  # Higher k = more sensitive to differences
    score = np.exp(-k * diff)
    
    return score


def compute_network_score(network_data, ref_metrics, weights):
    """
    Compute overall score for a network based on multiple criteria.
    
    Returns:
        dict with total_score and individual component scores
    """
    G = network_data['graph']
    metrics = network_data['metrics']
    syntax = network_data['syntax_metrics']
    
    # Extract network metrics
    area_km2 = (WINDOW_SIZE_M / 1000.0) ** 2
    node_density = G.number_of_nodes() / area_km2
    avg_degree = metrics['avg_degree']
    avg_segment_length = metrics['avg_segment_length']
    intelligibility = syntax['intelligibility']
    mean_depth = syntax['avg_mean_depth']
    
    # Compute individual scores
    scores = {
        'node_density': compute_similarity_score(node_density, ref_metrics['node_density']),
        'avg_degree': compute_similarity_score(avg_degree, ref_metrics['avg_degree']),
        'avg_segment_length': compute_similarity_score(avg_segment_length, ref_metrics['avg_segment_length']),
        'intelligibility': compute_similarity_score(intelligibility, ref_metrics['intelligibility']),
        'mean_depth': compute_similarity_score(mean_depth, ref_metrics['mean_depth'])
    }
    
    # Compute weighted total score
    total_score = sum(scores[metric] * weights[metric] for metric in scores.keys())
    
    return {
        'total_score': total_score,
        'component_scores': scores,
        'actual_values': {
            'node_density': node_density,
            'avg_degree': avg_degree,
            'avg_segment_length': avg_segment_length,
            'intelligibility': intelligibility,
            'mean_depth': mean_depth
        }
    }


# Compute scores for all networks
print("Computing scores for all 20 networks...\n")
print("="*70)

for network_data in generated_networks:
    score_data = compute_network_score(network_data, ref_metrics, weights)
    network_data['ranking'] = score_data
    
    net_id = network_data['id']
    total = score_data['total_score']
    
    print(f"Network {net_id+1:2d}: Total Score = {total:.4f}")

print("="*70)
print("\n✓ Scoring complete")

## Rank Networks

In [ ]:
# Sort networks by total score (descending)
ranked_networks = sorted(generated_networks, key=lambda x: x['ranking']['total_score'], reverse=True)

# Add rank to each network
for rank, network in enumerate(ranked_networks, 1):
    network['rank'] = rank

# Display top 10
print("\n" + "="*70)
print(" "*25 + "TOP 10 NETWORKS")
print("="*70)
print(f"{'Rank':<6} {'Network':<10} {'Score':<10} {'Nodes':<8} {'Edges':<8} {'Intelligibility':<15}")
print("-"*70)

for network in ranked_networks[:10]:
    rank = network['rank']
    net_id = network['id'] + 1
    score = network['ranking']['total_score']
    nodes = network['graph'].number_of_nodes()
    edges = network['graph'].number_of_edges()
    intell = network['ranking']['actual_values']['intelligibility']
    
    print(f"{rank:<6} {net_id:<10} {score:<10.4f} {nodes:<8} {edges:<8} {intell:<15.4f}")

print("="*70)

# Select best network
best_network = ranked_networks[0]
print(f"\n🏆 BEST NETWORK: Network {best_network['id']+1}")
print(f"   Score: {best_network['ranking']['total_score']:.4f}")
print(f"   Nodes: {best_network['graph'].number_of_nodes()}")
print(f"   Edges: {best_network['graph'].number_of_edges()}")

## Detailed Analysis of Best Network

In [ ]:
best = best_network

print("\n" + "="*70)
print(" "*20 + "BEST NETWORK DETAILED ANALYSIS")
print("="*70)

print(f"\nNetwork ID: {best['id']+1}")
print(f"Overall Rank: {best['rank']}")
print(f"Total Score: {best['ranking']['total_score']:.4f}")

print("\n" + "-"*70)
print("METRIC COMPARISON (Actual vs Target)")
print("-"*70)
print(f"{'Metric':<25} {'Target':<12} {'Actual':<12} {'Score':<10} {'Weight':<8}")
print("-"*70)

for metric in ref_metrics.keys():
    target = ref_metrics[metric]
    actual = best['ranking']['actual_values'][metric]
    score = best['ranking']['component_scores'][metric]
    weight = weights[metric]
    
    print(f"{metric:<25} {target:<12.3f} {actual:<12.3f} {score:<10.4f} {weight:<8.2f}")

print("="*70)

# Component score breakdown
print("\nWEIGHTED COMPONENT CONTRIBUTIONS:")
print("-"*70)
for metric in ref_metrics.keys():
    score = best['ranking']['component_scores'][metric]
    weight = weights[metric]
    contribution = score * weight
    print(f"  {metric:<25}: {contribution:.4f} (score: {score:.4f} × weight: {weight:.2f})")
print(f"  {'TOTAL':<25}: {best['ranking']['total_score']:.4f}")
print("="*70)

## Visualize Ranking Results

In [ ]:
# Create comprehensive ranking visualization
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Overall Score Distribution
ax1 = fig.add_subplot(gs[0, :])
scores = [net['ranking']['total_score'] for net in ranked_networks]
network_ids = [net['id']+1 for net in ranked_networks]
colors = ['gold' if i == 0 else 'steelblue' for i in range(len(scores))]

bars = ax1.bar(range(1, 21), scores, color=colors, alpha=0.7, edgecolor='black', linewidth=1)
ax1.set_xlabel('Network (ranked by score)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Total Score', fontsize=11, fontweight='bold')
ax1.set_title('Network Ranking: Overall Scores (Gold = Best)', fontsize=13, fontweight='bold')
ax1.set_xticks(range(1, 21))
ax1.set_xticklabels([ranked_networks[i]['id']+1 for i in range(20)])
ax1.grid(axis='y', alpha=0.3)
ax1.axhline(best['ranking']['total_score'], color='red', linestyle='--', linewidth=2, alpha=0.5)

# Add value labels on top 5
for i in range(5):
    height = bars[i].get_height()
    ax1.text(bars[i].get_x() + bars[i].get_width()/2., height,
            f'{scores[i]:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# 2-6. Component Score Comparisons
metric_names = list(ref_metrics.keys())
positions = [(1, 0), (1, 1), (1, 2), (2, 0), (2, 1)]

for idx, (metric, pos) in enumerate(zip(metric_names, positions)):
    ax = fig.add_subplot(gs[pos[0], pos[1]])
    
    # Get actual values for all networks
    values = [net['ranking']['actual_values'][metric] for net in ranked_networks[:10]]
    net_labels = [net['id']+1 for net in ranked_networks[:10]]
    target = ref_metrics[metric]
    
    # Bar plot
    colors = ['gold' if i == 0 else 'lightblue' for i in range(len(values))]
    ax.barh(range(len(values)), values, color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)
    
    # Target line
    ax.axvline(target, color='red', linestyle='--', linewidth=2, label=f'Target: {target:.2f}')
    
    ax.set_yticks(range(len(values)))
    ax.set_yticklabels(net_labels)
    ax.set_xlabel('Value', fontsize=9)
    ax.set_ylabel('Network ID', fontsize=9)
    ax.set_title(f'{metric.replace("_", " ").title()}\n(Top 10)', fontsize=10, fontweight='bold')
    ax.legend(fontsize=8)
    ax.grid(axis='x', alpha=0.3)
    ax.invert_yaxis()

# 7. Score vs Intelligibility scatter
ax7 = fig.add_subplot(gs[2, 2])
scores_all = [net['ranking']['total_score'] for net in generated_networks]
intell_all = [net['ranking']['actual_values']['intelligibility'] for net in generated_networks]

ax7.scatter(intell_all, scores_all, s=100, alpha=0.6, c='steelblue', edgecolors='black', linewidths=0.5)
ax7.scatter([best['ranking']['actual_values']['intelligibility']], [best['ranking']['total_score']],
           s=200, c='gold', marker='*', edgecolors='black', linewidths=2, label='Best Network', zorder=5)
ax7.set_xlabel('Intelligibility', fontsize=10, fontweight='bold')
ax7.set_ylabel('Total Score', fontsize=10, fontweight='bold')
ax7.set_title('Score vs Intelligibility', fontsize=11, fontweight='bold')
ax7.legend(fontsize=9)
ax7.grid(alpha=0.3)

plt.suptitle(f'Network Ranking Analysis - Best: Network {best["id"]+1} (Score: {best["ranking"]["total_score"]:.4f})',
            fontsize=16, fontweight='bold', y=0.995)

# Save
plt.savefig('outputs/generated/visualizations/E1_network_ranking.svg',
           format='svg', bbox_inches='tight', dpi=300)
print("Saved: outputs/generated/visualizations/E1_network_ranking.svg")

plt.show()

## Visualize Best Network

In [ ]:
# Visualize the best network
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

G = best['graph']
pos = best['pos']

# Window boundary
ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,
                       fill=False, edgecolor='black', linestyle='-', linewidth=2))

# Draw edges
for u, v in G.edges():
    x = [pos[u][0], pos[v][0]]
    y = [pos[u][1], pos[v][1]]
    ax.plot(x, y, color='steelblue', linewidth=2, alpha=0.7, zorder=1)

# Draw nodes colored by degree
degrees = dict(G.degree())
max_degree = max(degrees.values()) if degrees else 1

for node in G.nodes():
    degree = degrees[node]
    color_val = degree / max_degree
    node_color = plt.cm.RdYlBu_r(color_val)
    size = 50 + degree * 30
    ax.scatter(pos[node][0], pos[node][1], s=size, c=[node_color],
              zorder=2, edgecolors='black', linewidths=1)

ax.set_xlim(-20, WINDOW_SIZE_M + 20)
ax.set_ylim(-20, WINDOW_SIZE_M + 20)
ax.set_aspect('equal')

title = f"BEST NETWORK: #{best['id']+1} (Rank 1 / 20)\n"
title += f"Score: {best['ranking']['total_score']:.4f} | "
title += f"Nodes: {G.number_of_nodes()} | Edges: {G.number_of_edges()} | "
title += f"Intelligibility: {best['ranking']['actual_values']['intelligibility']:.3f}"

ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('X (meters)', fontsize=11)
ax.set_ylabel('Y (meters)', fontsize=11)
ax.grid(True, alpha=0.3)

# Save
plt.savefig('outputs/generated/visualizations/E2_best_network.svg',
           format='svg', bbox_inches='tight', dpi=300)
print("Saved: outputs/generated/visualizations/E2_best_network.svg")

plt.show()

## Save Selected Network

In [ ]:
# Save best network for building generation
with open('outputs/generated/selected/best_network.pkl', 'wb') as f:
    pickle.dump(best_network, f)

print("✓ Saved best network to: outputs/generated/selected/best_network.pkl")

# Save ranking table
ranking_data = []
for net in ranked_networks:
    ranking_data.append({
        'Rank': net['rank'],
        'Network_ID': net['id'] + 1,
        'Total_Score': net['ranking']['total_score'],
        'Nodes': net['graph'].number_of_nodes(),
        'Edges': net['graph'].number_of_edges(),
        'Node_Density': net['ranking']['actual_values']['node_density'],
        'Avg_Degree': net['ranking']['actual_values']['avg_degree'],
        'Avg_Segment_Length': net['ranking']['actual_values']['avg_segment_length'],
        'Intelligibility': net['ranking']['actual_values']['intelligibility'],
        'Mean_Depth': net['ranking']['actual_values']['mean_depth']
    })

df_ranking = pd.DataFrame(ranking_data)
df_ranking.to_csv('outputs/generated/selected/network_rankings.csv', index=False)
print("✓ Saved ranking table to: outputs/generated/selected/network_rankings.csv")

print("\n" + "="*70)
print("✓ STEP 5 COMPLETE!")
print("="*70)
print(f"\nSelected Network: #{best_network['id']+1}")
print(f"Score: {best_network['ranking']['total_score']:.4f}")
print(f"\nNext: Generate optimized buildings for this network in final step.")

## Summary

**Step 5 Complete!**

Results:
- Ranked all 20 networks using multi-objective scoring
- Criteria: node density, avg degree, segment length, intelligibility, mean depth
- Selected best network based on similarity to reference city (London)
- Saved selected network for final building generation

**Next Steps:**
1. Generate optimized building footprints for selected network
2. Match building distributions from reference city
3. Export final result to GeoJSON/Shapefile